# Mini Proyecto Datos Masivos III

- Barrero Olguín Adolfo Patricio
- Martínez Ostoa Néstor Iván
- Ramírez Bondi Jorge Alejandro

In [ ]:
!pip install pyspark
!pip install wordcloud
!pip install -U -q PyDrive
!pip install boto3
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 133kB 3.2MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
openjdk-8-jdk-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [1]:
# Cosas importantes de pyspark
import pyspark
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import size

# Extras
from operator import add, sub
from time import sleep
import string

from urllib.request import urlopen
import json
import boto3

import numpy as np
import matplotlib.pyplot as plt

from wordcloud import WordCloud
import re
import nltk
nltk.download('stopwords')

from scipy.sparse import csr_matrix
from sklearn.preprocessing import OrdinalEncoder

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nestorivanmo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nestorivanmo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# AWS

In [4]:
comprehend = boto3.client(
    service_name='comprehend', region_name='us-east-1',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    aws_session_token=aws_session_token
)

In [5]:
def analyze_tweet(text):
  return comprehend.detect_sentiment(Text=text, LanguageCode='es')['Sentiment']

In [6]:
analyze_tweet("RT @claraluzflores: Este 6 de junio el poder es tuyo, juntos vamos a construir ese puente al futuro que tanto necesitamos. \n\nVamos a regres…")

'POSITIVE'

# Lectura Datos

In [7]:
url = "https://raw.githubusercontent.com/nestorivanmo/twitter-mexican-elections/main/data_nestor/data.json?token=AHLLFMVYMEEMFMQ22XXSDN3AZDXJ2"
response = urlopen(url)
data_json = json.loads(response.read())
type(data_json)

dict

In [8]:
datos_lista = []
for i in data_json:
    datos_lista.append(data_json[i])

In [9]:
np.random.seed(182)

batches = []

while len(datos_lista) > 100:
  batch_size = np.random.randint(50, 100)
  batch = datos_lista[:batch_size]
  datos_lista = datos_lista[batch_size:]
  batches.append(batch)
  
batches.append(datos_lista)

## Ejemplos de Tweets

In [ ]:
sc = SparkContext.getOrCreate()
delta_t = 1 # en segundos
ssc = StreamingContext(sc, delta_t)

flujo_rym = ssc.queueStream(batches)

flujo_rym.pprint(2)

ssc.start()
sleep(10)
ssc.stop(stopSparkContext = False)

-------------------------------------------
Time: 2021-06-08 22:46:49
-------------------------------------------
{'data': {'entities': {'mentions': [{'start': 3, 'end': 18, 'username': 'claraluzflores'}]}, 'author_id': '1265700209792778240', 'referenced_tweets': [{'type': 'retweeted', 'id': '1398727240192610310'}], 'public_metrics': {'retweet_count': 8, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'source': 'Twitter Web App', 'text': 'RT @claraluzflores: Este 6 de junio el poder es tuyo, juntos vamos a construir ese puente al futuro que tanto necesitamos. \n\nVamos a regres…', 'id': '1398729426637033474', 'created_at': '2021-05-29T19:54:17.000Z'}, 'includes': {'users': [{'created_at': '2020-05-27T17:44:13.000Z', 'verified': False, 'name': 'mario perez b.', 'username': 'marioperezb2', 'description': '', 'id': '1265700209792778240', 'protected': False, 'public_metrics': {'followers_count': 19, 'following_count': 39, 'tweet_count': 1235, 'listed_count': 0}}]}, 'matching_rules': 

# Obtención del número de veces que una palabra es utilizada

In [ ]:
# Volvemos a generar la lista, para obtener los datos del texto por separado.
datos_lista = []
for i in data_json:
    datos_lista.append(data_json[i])

np.random.seed(182)

batches = []

while len(datos_lista) > 100:
  batch_size = np.random.randint(50, 100)
  batch = datos_lista[:batch_size]
  datos_lista = datos_lista[batch_size:]
  batches.append(batch)
  
batches.append(datos_lista)

# Procedemos a procesar el texto y contar la ocurrencia de las palabras.
palabras_vacias_extra = ['rt', '¿', '?', '¡', '!']
palabras_vacias = nltk.corpus.stopwords.words('spanish')
for p in palabras_vacias_extra:
  palabras_vacias.append(p)

def quita_palabras_vacias(texto):
  return [ palabra for palabra in texto if palabra not in palabras_vacias ]


def quita_caracteres_especiales(texto):
  return 


def actualiza_conteo(valores_nuevos, valor_actual):
  if valor_actual is None:
    valor_actual = 0
  return sum(valores_nuevos) + valor_actual


def agrupar_palabra(palabra):
  palabras = []
  for p in palabra[1]:
    p = ''.join(re.findall("\w", p))
    palabras.append((palabra[0], p))
  return palabras


sc = SparkContext.getOrCreate()
delta_t = 1 # en segundos
ssc = StreamingContext(sc, delta_t)
ssc.checkpoint("StreamingContext_checkpoint")

flujo_rym = ssc.queueStream(batches)

conteo_datos = flujo_rym.map(lambda tuit: (tuit['matching_rules'][0]['tag'],
                                           tuit['data']['text'].lower())) \
                        .map(lambda tuit: (tuit[0], tuit[1].split())) \
                        .map(lambda tuit: (tuit[0], quita_palabras_vacias(tuit[1]))) \
                        .map(lambda palabra: agrupar_palabra(palabra)) \
                        .flatMap(lambda x: x) \
                        .map(lambda x: (x, 1)) \
                        .updateStateByKey(actualiza_conteo)


conteo_datos.pprint()

ssc.start()
sleep(10)
ssc.stop(stopSparkContext = False)

## Dashboard

In [ ]:
from IPython.display import clear_output
import copy

In [ ]:
class TweetSource:
    def __init__(self, name):
        self.name = name
        self.positive = 0
        self.negative = 0
        self.neutral = 0
        
    def add(self, sentiment, count):
        if sentiment == 'POSITIVE':
            self.positive += count
        elif sentiment == 'NEGATIVE':
            self.negative += count
        else:
            self.neutral += count

class DashBoard:
    def __init__(self):
        self.rdd = None
        self.iOS_source     = TweetSource(name='iOS')
        self.android_source = TweetSource(name='Android')
        self.other_source   = TweetSource(name='Other')

    def add_rdd(self, rdd):
        self.rdd = rdd

    def process_sources_from(self, record):
        source = record[0][0].lower()
        sentiment = record[0][1]
        count = record[1]
        if 'iPhone'.lower() in source or 'iPad'.lower() in source:
            self.iOS_source.add(sentiment, count)
        elif 'Android'.lower() in source:
            self.android_source.add(sentiment, count)
        else:
            self.other_source.add(sentiment, count)

    def show_tweet_sources(self):
        print ("{:<12} {:<16} {:<16} {:<16}".format('iOS', self.iOS_source.positive, self.iOS_source.negative, self.iOS_source.neutral))
        print ("{:<12} {:<16} {:<16} {:<16}".format('Android', self.android_source.positive, self.android_source.negative, self.android_source.neutral))
        print ("{:<12} {:<16} {:<16} {:<16}".format('Otro', self.other_source.positive, self.other_source.negative, self.other_source.neutral))

    def view_tweet_sources(self, candidate):
        if self.rdd is None:
            raise Exception("Dashboard must contain an RDD")
        self.rdd = self.rdd.filter(lambda x: x['matching_rules'][0]['tag'] == candidate) \
                        .map(lambda x: ((x['data']['source'], x['sentiment']), 1)) \
                            .reduceByKey(add)

        for record in self.rdd.collect():
            self.process_sources_from(record)
        self.show_tweet_sources()
        print("")

In [ ]:
d1 = DashBoard()
d2 = DashBoard()
d3 = DashBoard()
d4 = DashBoard()

politicians = {
    'Samuel Garcia': 'Movimiento Ciudadano',
    'Clara Luz': 'Morena',
    'Adrian de la Garza': 'PRI',
    'Fernando Larrazabal': 'PAN',
}

def view_source_per_candidate(candidate, dashboard, rdd):
    print(f"{candidate} - {politicians[candidate]}")
    print("{:<12} {:<15} {:<15} {:<15}".format('', 'POSITIVO', 'NEGATIVO', 'NEUTRAL'))
    dashboard.add_rdd(rdd)
    dashboard.view_tweet_sources(candidate)


def update_dashboard(time, rdd):
    global d1, d2, d3, d4
    clear_output(wait=True)
    print("------------------------------------------------------")
    print(f"Tiempo: {time}")
    print("------------------------------------------------------")
    view_source_per_candidate('Samuel Garcia', d1, rdd)
    #view_source_per_candidate('Clara Luz', d2, rdd)
    #view_source_per_candidate('Adrian de la Garza', d3, rdd)
    #view_source_per_candidate('Fernando Larrazabal', d4, rdd)

In [ ]:
sc = SparkContext.getOrCreate()
delta_t = 1 # en segundos
ssc = StreamingContext(sc, delta_t)

tweets_stream = ssc.queueStream(batches).foreachRDD(update_dashboard)

ssc.start()
sleep(10)
ssc.stop(stopSparkContext = False)

------------------------------------------------------
Tiempo: 2021-06-09 01:38:46
------------------------------------------------------
Samuel Garcia - Movimiento Ciudadano
             POSITIVO        NEGATIVO        NEUTRAL        
iOS          1                43               67              
Android      7                80               127             
Otro         1                12               18              



# Mapa de Palabras

In [10]:
%matplotlib

Using matplotlib backend: MacOSX


In [11]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nestorivanmo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
stop_words_sp = set(stopwords.words('spanish')).union({"rt"})

In [13]:
def quita_palabras_vacias(texto, p = stop_words_sp):
    texto_limpio = [ palabra for palabra in texto if palabra not in p ]
    return texto_limpio

In [18]:
palabras = ""

def init_mapa():
    global palabras
    
    palabras = ""
    wc = WordCloud()
    wc.generate("Hola")
    plt.ion()
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.axis("off")

    
    mapa = ax.imshow(wc, interpolation='bilinear')
    return fig, mapa

def actualizar_mapa(nuevas_palabras, fig, mapa, nombre = "a.png"):
    global palabras
    palabras = palabras + " " + nuevas_palabras
    print(len(palabras))

    wc = WordCloud()
    wc.generate(palabras)
    mapa.set_data(wc)
    fig.canvas.draw()
    fig.canvas.flush_events()
    fig.savefig(nombre)
    sleep(2)

In [19]:
fig, mapa = init_mapa()
usernames = ["samuelgarcias","adriandelagarza","andreac","claraluzflores","crisitibellaka","greyszu","mariodelgado","ferlarrazabalnl","michrmzd"]

sc = SparkContext.getOrCreate()
delta_t = 1 # en segundos
ssc = StreamingContext(sc, delta_t)

flujo = ssc.queueStream(batches)

flujo.map(lambda x : x["data"]["text"]) \
     .map(lambda dialogo: dialogo.lower()) \
     .map(lambda dialogo: dialogo.translate(str.maketrans('', '', string.punctuation + string.digits + "¿¡"))) \
     .map(lambda dialogo: dialogo.split(' ')) \
     .map(lambda palabra: quita_palabras_vacias(palabra)) \
     .map(lambda palabra: quita_palabras_vacias(palabra, usernames)) \
     .flatMap(lambda x: x) \
     .map(lambda dialogo: dialogo.translate(str.maketrans('', '', string.whitespace))) \
     .filter(lambda x : len(x) > 0) \
     .foreachRDD(lambda time, rdd : actualizar_mapa(" ".join(rdd.collect()), fig, mapa, "palabras_%s.png" % (str(time))))

ssc.start()
sleep(10)
ssc.stop(stopSparkContext = False)

6731
10862
17224
